<h4> Import libraries to use </h4>

In [1]:
import itertools
import numpy as np
import pandas as pd

<h4> methods to find impurity of a node </h4>

In [2]:
def gini_impurity(y):
    """_finds the gini impurity of a node. Measure of impurity in the system_
    Args:
        y (_Pandas Series_): _all our class labels_
    Returns:
        _float_: _entropy ranges from 0(purest) to 1(most impure)._
    """
    if isinstance(y, pd.Series):
        p = y.value_counts()/y.shape[0]
        gini = 1-np.sum(p**2)
        return(gini)
    else:
        raise('Object must be a Pandas Series.')

In [3]:
def entropy(y):
    """_finds the entropy of a node. Measure of impurity in the system_

    Args:
        y (_Pandas Series_): _all our class labels_

    Returns:
        _float_: _gini index ranges from 0(purest) to 1(most impure)._
    """    
    if isinstance(y, pd.Series):
        a = y.value_counts()/y.shape[0]
        entropy = np.sum(-a*np.log2(a+1e-9))
        return(entropy)
    else:
        raise('Object must be a Pandas Series.')

<h4> methods to choose cuts for decision tree </h4>

In [4]:
def variance(y):
    """_finds variance if column is numerical_

    Args:
        y (_Pandas Series_): _column with predictors_

    Returns:
        _float_: _variance of the column._
    """
    if(len(y) == 1):
        return 0
    else:
        return y.var()

In [5]:
def information_gain(y, filter_column, func=entropy):
    """_get information gain from parent to child nodes
    
    Args:
        y (_Pandas Series_): _labels in current node_
        filter_column (_Pandas Series_): subset of training data on which to split node
        func (_function_): function to find impurity of node
        
    Returns:
        _float_: _this is the information gain. between 0 and 1
    
    """
    a = sum(filter_column)
    b = filter_column.shape[0] - a
  
    if(a == 0 or b ==0):
        # if either is 0, it means only 1 child has been created that is a replica of the parent
        # thus there is no information gain
        information_gain = 0
  
    else:
        if y.dtypes != 'O':
            # numerical columns
            information_gain = variance(y) - (a/(a+b)* variance(y[filter_column])) - (b/(a+b)*variance(y[-filter_column]))
        else:
            # categorical columns
            information_gain = func(y)-a/(a+b)*func(y[filter_column])-b/(a+b)*func(y[-filter_column])
  
    return information_gain

<h4> methods to calculate the best split for a variable </h4>

In [6]:
def categorical_options(x_column):
    """_find all the possible combinations in categorical column
   this is for the random aspect during the calculation of best split
   
   Args:
       x_column (_Pandas Series_): categorical column on which to find subsets
    
    Returns:
        options (_list_): list of lists with combinations
   
   """
    x_column = x_column.unique()

    options = []
    for L in range(0, len(x_column)+1):
        for subset in itertools.combinations(x_column, L):
            subset = list(subset)
            options.append(subset)
    # first element in options is empty, last is not subset
    return options[1:-1]

In [7]:
def max_information_gain_split(x, y, func=entropy):
    """_find the max information gain split
    
    Args:
        x (_Pandas Series): _predictor variable_
        y (_Pandas Series): _target variable_
        func: function to be used for the best split
    
    Return:
        tuple: best_split(value), information gain for that split, type of variable(numeric/categorical), whether or not there was information gain
    """
    split_value = []
    information_gains = [] 

    numeric_variable = True if x.dtypes != 'O' else False

  # Create options according to variable type
    if numeric_variable:
        options = x.sort_values().unique()[1:]
    else: 
        options = categorical_options(x)

    # Calculate ig for all values
    for val in options:
        filter_column =   x < val if numeric_variable else x.isin(val)
        val_ig = information_gain(y, filter_column, func)
        # Append results
        information_gains.append(val_ig)
        split_value.append(val)

  # Check if there are more than 1 results if not, return False
    if len(information_gains) == 0:
        return(None,None,None, False)

    else:
  # Get results with highest IG
        best_ig = max(information_gains)
        best_ig_index = information_gains.index(best_ig)
        best_split = split_value[best_ig_index]
        return(best_ig,best_split,numeric_variable, True)



<h4> methods to get best split, make the split and make a prediction </h4>

In [8]:
def get_best_split(name, data):
    """_get the best split of a node_

    Args:
        y (_string_): _name of target variable_
        data (_Pandas DataFrame_): _training data_

    Returns:
        _tuple_: _variable with highest information gain, value on which to split variable,
                    value of the information gain, whether variable is numerical or categorical_
    """    
    best_split_df = data.drop(name, axis= 1).apply(max_information_gain_split, y = data[name])
    if sum(best_split_df.loc[3,:]) == 0:
        return(None, None, None, None)

    else:
        # Get only masks that can be splitted
        best_split_df = best_split_df.loc[:,best_split_df.loc[3,:]]

        # Get the results for split with highest Information Gain
        split_variable = best_split_df.iloc[0].astype(np.float32).idxmax()
        split_value = best_split_df[split_variable][1] 
        split_information_gain = best_split_df[split_variable][0]
        split_numeric = best_split_df[split_variable][2]

        return(split_variable, split_value, split_information_gain, split_numeric)
    
    
    

In [9]:
def make_split(variable, value, data, is_numeric):
    """_create 2 child nodes from parent node_

    Args:
        variable (_string_): _name of variable_
        data (_Pandas DataFrame_): _training data to be splitted_
        value (_int or float or string_): _value of variable to make the split_
        is_numeric (_bool_): is variable to be splitted numeric or not

    Returns:
        _tuple_: _2 child nodes, left node and right node_
    """ 
    if is_numeric:
        left_node = data[data[variable] < value]
        right_node = data[(data[variable] < value) == False]

    else:
        left_node = data[data[variable].isin(value)]
        right_node = data[(data[variable].isin(value)) == False]

    return(left_node, right_node)

In [11]:
def make_prediction(data, is_numeric):
    """_make prediction._

    Args:
        data (_Pandas DataFrame_): _training data to be splitted_
        is_numeric (_bool_): is variable numeric or categorical

    Returns:
        _prediction_: _prediction for tree_
    """ 
    if is_numeric:
        pred = data.value_counts().idxmax()
    else:
        pred = data.mean()

    return pred

<h4> train our decision tree </h4>

In [12]:
def train_tree(data,y, is_numeric, max_depth = None,min_samples_split = None, min_information_gain = 1e-20, counter=0, max_categories = 20):
    """_train a Decision Tree_

    Args:
        data (_Pandas DataFrame_): _training data for the Decision Tree_
        y (_string_): _name of target varible_
        is_numeric (_bool_): is target variable categorical or numeric
        max_depth (_int_): depth on which to stop splitting
        min_samples_split (_int_): number of samples below which a new child node cant be created
        min_information_gain (_float_): smallest amount of information gain for a split to be considered valid
        max_categories (_int_): max number of categories in a categorical column. Many unique items cause high computation time and slow learning
        counter (int): keeps count of depth which is also the number of recursions
        
    Returns:
        _subtree_: _this is a set of rules used to predict_
    """    
    if counter==0:
        # on first iteration, check each categorical column
        # if any column has more unique values than max_categories, raise error
        types = data.dtypes
        # categorical columns
        check_columns = types[types == "object"].index
        for column in check_columns:
            # number of unique values in column
            var_length = len(data[column].value_counts()) 
            if var_length > max_categories:
                raise ValueError('The variable ' + column + ' has '+ str(var_length) + ' unique values, which is more than the accepted ones: ' +  str(max_categories))
  
    # Check for depth conditions on every recursion iteration
    if max_depth == None:
        depth_cond = True
    else:
        if counter < max_depth:
            depth_cond = True
        else:
            depth_cond = False
    # Check for sample conditions on every recursion iteration
    if min_samples_split == None:
        sample_cond = True
    else:
        if data.shape[0] > min_samples_split:
            sample_cond = True
        else:
            sample_cond = False
    # if stopping conditions are not broken
    if depth_cond & sample_cond:
        split_variable, split_value, split_information_gain, split_numeric = get_best_split(y, data)
    # If ig condition is fulfilled, make split 
        if split_information_gain is not None and split_information_gain >= min_information_gain:
            counter += 1  # on every
            left,right = make_split(split_variable, split_value, data, split_numeric)
      # document the tree
            split_type = "<=" if split_numeric else "in"
            question =   "{} {}  {}".format(split_variable,split_type,split_value)
            subtree = {question: []}
      # Find answers (recursion)
            yes_answer = train_tree(left,y, target_type, max_depth,min_samples_split,min_information_gain, counter)
            no_answer = train_tree(right,y, target_type, max_depth,min_samples_split,min_information_gain, counter)
            if yes_answer == no_answer: # will only happen at leaf node
                subtree = yes_answer
            else:
                subtree[question].append(yes_answer)
                subtree[question].append(no_answer)
    # If it doesn't match IG condition, make prediction
        else:
            pred = make_prediction(data[y],is_numeric)
            return pred
   # Drop dataset if doesn't match depth or sample conditions
    else:
        pred = make_prediction(data[y],is_numeric)
        return pred
    return subtree


<h4> method to make prediction </h4>

In [13]:
def classify_regress(datapoint, subtree):
    """_makes predictions, classification or regression_
    
    Args:
        datapoint (_numpy row_): single instance to be predicted
        subtree (_Decision Tree_): trained tree
        
    Returns:
        answer (_type_): a prediction for the datapoint
    """
    # get subtree of decisions
    question = list(subtree.keys())[0]
    
    # check whether variable type is numerical or categorical
    if question.split()[1] == '<=':
        if datapoint[question.split()[0]] <= float(question.split()[2]):
            answer = subtree[question][0]
        else:
            answer = subtree[question][1]
    else:
        if datapoint[question.split()[0]] in (question.split()[2]):
            answer = subtree[question][0]
        else:
            answer = subtree[question][1]
    
    # if the answer is not a dictionary means we've reached a leaf node
    if not isinstance(answer, dict):
        return answer
    else:
        residual_tree = answer
        return classify_regress(datapoint, residual_tree)

<h4> Predict sample dataset with decision tree </h4>

In [15]:
titanic = pd.read_csv('titanic.csv')
titanic.head()

FileNotFoundError: [Errno 2] No such file or directory: 'titanic.csv'

In [ ]:
titanic_lite = titanic.loc[:,["Embarked", "Age", "Fare", "Survived"]]
titanic_lite = titanic_lite.loc[titanic_lite.isna().sum(axis=1) == 0, :]
titanic_tree = train_tree(titanic_lite, 'Survived', True, max_depth=10, min_samples_split=10)
titanic_tree

In [ ]:
titanic_prediction = []
num_obs = 20

for i in range(num_obs):
    obs_pred = classify_regress(titanic_lite.iloc[i,:], titanic_tree)
    titanic_prediction.append(obs_pred)

print("Predictions: ", titanic_prediction,
      "\n\n Real values:", titanic_lite.Survived[:num_obs].to_numpy())

In [ ]:
type(titanic_tree)

In [ ]:
titanic_tree

In [ ]:
titanic_tree['Fare <=  52.5542'][1]#['Fare <=  10.5'][0]['Age <=  33.0'][0]["Embarked in  ['S']"][0]['Age <=  31.0'][0]

In [ ]:
titanic_tree.keys()

In [ ]:
v1 = titanic_tree['Fare <=  52.5542']
v1

In [ ]:
len(v1)

In [ ]:
v1[0]

In [ ]:
v1[1]

In [ ]:
v1[0].keys()

In [ ]:
v1[1].keys()

In [ ]:
v2 = v1[0]['Fare <=  10.5']
v2

In [ ]:
v22 = v1[1]['Age <=  64.0']
v22

In [ ]:
len(v22)

In [ ]:
v2[0].keys()

In [ ]:
v3 = v2[0]['Age <=  33.0']
v3

In [ ]:
v3[0].keys()

In [ ]:
v4 = v3[0]["Embarked in  ['S']"]
v4

In [ ]:
v4[0].keys()

In [ ]:
v5 = v4[0]['Age <=  31.0']
v5

<h3> random forest </h3> 
<h4> method to get a bootstrap of from dataset </h4>

In [ ]:
def draw_bootstrap(x):
    """_gets a bootstrap of all the samples
    
    Args:
        x (_Pandas DataFrame_): _training dataset with labels_
    
    Return:
        _x_: a bootstrapped dataset
    """
    n_samples = x.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return x.iloc[idxs, :]

In [ ]:
def random_forest(data, y, is_numeric, n_trees=100):
    """_train a random fores_

    Args:
        data (_Pandas DataFrame_): _training data for the Decision Tree_
        y (_string_): _name of target varible_
        is_numeric (_bool_): is target variable categorical or numeric
        n_trees (_int_): number of trees in forest
        
    Returns:
        _trees_: _list of decision trees trained_
    """    
    trees = [] # list of trees 
    for _ in range(n_trees):
        x_samples = draw_bootstrap(data)
        tree = train_tree(data,y, is_numeric, max_depth = None,min_samples_split = None, min_information_gain = 1e-20, counter=0, max_categories = 20)
        trees.append(tree)
    return trees


In [ ]:
def most_common_label(trees, is_numeric, X):
    """finds the most common label in the forest
    
    Args:
        trees (_list_): a list of trained trees (forest)
        is_numeric (_bool_): classifier or regressor
        X (_Pandas DataFrame_): training data without the label
        
    Return:
        most_common (_int/float_): the most common answer
    """
    if is_numeric:
        pass
    else:
        num_obs = X.shape(0)
        predictions = []
        for tree in trees:
            tree_pred = [classify_regress(X.iloc[i,:], tree) for i in range(num_obs)]
            predictions.append(tree_pred)
    predictions = np.array(predictions)
    most_common = np.round(np.mean(predictions, axis=0), 0)
    return most_common

In [ ]:
titanic = pd.read_csv('titanic.csv')

In [ ]:
am = np.random.choice(15, 15, replace=True)
k.iloc[am, :]

In [ ]:
j.iloc[am]

In [ ]:
l = draw_bootstrap(x=k, y =j)

In [ ]:
k = titanic.head(15)
j = titanic.head(15)
k = k.loc[:, ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age']]
j = j.loc[:, 'Survived']

In [ ]:
k.columns

In [ ]:
pd.DataFrame(l)

In [ ]:
l